# Lista 4

In [1]:
import numpy as np

## Exercício 1

Implementar método Francis. Utilizar matrizes simétricas com dimensões $n=10, 100, 500, 1000$.

In [ ]:
def FrancisMethod(n, Max_It, Tol):
    
    # Definir matriz randomica de dimensão n
    # Transformar em Simetrica
    k = 1
    E = 0
    A = np.zeros(shape=(n,n))
    for i in range (n):
        for j in range (n):
            A[i,j]=A[j,i]= np.random.randint(0,100)
    return A

    while(E > Tol and k < Max_It):              # Enquanto E > TOL e k < MAX_IT
        Q, R = np.linalg.qr(A)                      # A0 = Calcular Fatoração = QR
        A1 = R*Q                                   # Definir A1 = RQ
        #V1 = V 
        

    
        
        
        # V1 = fatoração
        # calcular E = det (Ak)

    return E